In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import os
PATH = '/content/drive/MyDrive/Parallel_PDE_project/fourier_neural_operator'
os.chdir(PATH)
import sys  
sys.path.insert(0, 'Notebooks')
from utils import UnitGaussianNormalizer
from FNO import FNO

In [54]:
#Parameters
ntrain = 1000
ntest = 1000
feature = 32
layers = 6
wavenumber = [25,25]
activ = 'relu' 
filename = 'FNO_problem_DarcyFlow_epoch_100_wavenumber_[25, 25]_features_32_act_relu_layers_6'
filename

'FNO_problem_DarcyFlow_epoch_100_wavenumber_[25, 25]_features_32_act_relu_layers_6'

In [55]:
################################################################
# Lifting map
################################################################
import torch.nn as nn
class R(nn.Module):
  def __init__(self, input, width):
    super().__init__()
    self.fc1 = nn.Linear(input, width)
  def forward(self,x):
    x = self.fc1(x)
    return x

In [56]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model= FNO(features_= feature, wavenumber= wavenumber, layers = layers, activation= activ).to(device)

In [57]:
model = torch.load(os.path.join('trained_models',filename+'.plt'), map_location=torch.device('cpu'))

In [58]:
model

FNO(
  (lifting): R(
    (fc1): Linear(in_features=3, out_features=32, bias=True)
  )
  (proj): Proj(
    (fc1): Linear(in_features=32, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=1, bias=True)
  )
  (fno): Sequential(
    (0): relu(FourierConv2d(32, 32, wavenumber=(25, 25)) + Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1)))
    (1): relu(FourierConv2d(32, 32, wavenumber=(25, 25)) + Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1)))
    (2): relu(FourierConv2d(32, 32, wavenumber=(25, 25)) + Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1)))
    (3): relu(FourierConv2d(32, 32, wavenumber=(25, 25)) + Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1)))
    (4): relu(FourierConv2d(32, 32, wavenumber=(25, 25)) + Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1)))
    (5): relu(FourierConv2d(32, 32, wavenumber=(25, 25)) + Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1)))
  )
)

In [46]:
from utils import MatReader, LpLoss

In [59]:
################################################################
# configs
################################################################
TRAIN_PATH = os.path.join('data', 'piececonst_r421_N1024_smooth1.mat')
TEST_PATH = os.path.join('data', 'piececonst_r421_N1024_smooth2.mat')

In [60]:
batch_size = 20
r_vec = [1,2,3,4,5]
h = [(r, int(((421 - 1)/r) + 1) ) for r in r_vec]
s_vec = h[::-1]

In [50]:
for r, s in s_vec:
  print(r, s)

5 85
4 106
3 141
2 211
1 421


In [61]:
test_losses_vec = []

In [62]:
myloss = LpLoss(size_average=False)
#y_normalizer.cuda()
for r, s in s_vec:
  reader = MatReader(TRAIN_PATH)
  x_train = reader.read_field('coeff')[:ntrain,::r,::r][:,:s,:s]
  y_train = reader.read_field('sol')[:ntrain,::r,::r][:,:s,:s]

  reader.load_file(TEST_PATH)
  x_test = reader.read_field('coeff')[:ntest,::r,::r][:,:s,:s]
  y_test = reader.read_field('sol')[:ntest,::r,::r][:,:s,:s]

  x_normalizer = UnitGaussianNormalizer(x_train)
  x_train = x_normalizer.encode(x_train)
  x_test = x_normalizer.encode(x_test)

  y_normalizer = UnitGaussianNormalizer(y_train)
  y_train = y_normalizer.encode(y_train)

  x_train = x_train.reshape(ntrain,s,s,1)
  x_test = x_test.reshape(ntest,s,s,1)

  test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

  model.eval()
  test_l2 = 0.0
  with torch.no_grad():
      for x, y in test_loader:
          x, y = x.to(device), y.to(device)

          out = model(x).reshape(batch_size, s, s)
          #out = model(x).reshape(batch_size, s)
          out = y_normalizer.decode(out)

          test_l2 += myloss(out.view(batch_size,-1), y.view(batch_size,-1)).item()

  test_l2 /= ntest
  test_losses_vec.append(test_l2)

  print(test_l2)

0.011869428619742394
0.01916393032670021
0.036846539437770845
0.05604379880428314


OSError: ignored